So, first thing first. Ill start examining a radom point of New York, and see how dense is that area with a query to Foursquare API. The expected result is that New York its a totally hot area terribly expensive, and as such, not a safe bet for someone who is taking its first steps in the international scene. High risk high reward surely, but it has no meaning to throw yourself to the sharks like that, and ill try to prove it through data:

In [ ]:
#Import necesary libraries for the sake of this particular study:



In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Folium installed
Libraries imported.


In [2]:
#Ill save the keys necesary to call the Foursquare API later, as shown in the course:
CLIENT_ID = '0ETFVTQEUWSRRP2EQ5JHZW0UUC4AMRXYX0YFFHQAKZLJJD10' # your Foursquare ID
CLIENT_SECRET = 'T4K2MZTXLDEVIPIL15LTDI3QPNBSHMYAIHXBCLNBSFFEH1FH' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 52
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 0ETFVTQEUWSRRP2EQ5JHZW0UUC4AMRXYX0YFFHQAKZLJJD10
CLIENT_SECRET:T4K2MZTXLDEVIPIL15LTDI3QPNBSHMYAIHXBCLNBSFFEH1FH


In [3]:
#As a random point in a relevant area, i will reutilize the geolocator to the exact area as shown in the course:
address = '102 North End Ave, New York, NY'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address, timeout=10)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

40.7149555 -74.0153365


In [89]:
#Ill start the query using a ratio of 2500, expecting the result to surpass the 49 unfortunately limit of the API, using as keyword 'pizza':
search_query = 'pizza'
radius = 10000
print('done!')

done!


In [190]:
#Call the API:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=0ETFVTQEUWSRRP2EQ5JHZW0UUC4AMRXYX0YFFHQAKZLJJD10&client_secret=T4K2MZTXLDEVIPIL15LTDI3QPNBSHMYAIHXBCLNBSFFEH1FH&ll=40.36,-74.01&v=20180604&query=bank&radius=5000&limit=52'

In [197]:
#Request the information and display it properly:
results = requests.get(url).json()

In [192]:
#Normalize the info as shown over the course and transform it to a dataframe, displaying the first 3 results:
venues = results['response']['venues']
dataframe = json_normalize(venues)
dataframe.head(3)

,categories,delivery.id,delivery.provider.icon.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.name,delivery.url,hasPerk,id,location.address,...,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d13d941735', 'name': 'H...",NaN,NaN,NaN,NaN,NaN,NaN,False,4be16a84d816c928d5f7efd9,101 Ridge Rd,...,"[101 Ridge Rd, Little Silver, NJ 07739, United...","[{'label': 'display', 'lat': 40.35016004059454...",40.350160,-74.039729,NaN,07739,NJ,Red Bank Regional High School,v-1549514468,NaN
1,"[{'id': '52f2ab2ebcbc57f1066b8b46', 'name': 'S...",NaN,NaN,NaN,NaN,NaN,NaN,False,4b9a41e4f964a520c6a735e3,362 Broad St,...,"[362 Broad St, Red Bank, NJ 07701, United States]","[{'label': 'display', 'lat': 40.339588, 'lng':...",40.339588,-74.064127,NaN,07701,NJ,Super Foodtown of Red Bank,v-1549514468,209529457
2,"[{'id': '4bf58dd8d48988d129951735', 'name': 'T...",NaN,NaN,NaN,NaN,NaN,NaN,False,4af38fc5f964a5209fee21e3,175 Monmouth St.,...,"[175 Monmouth St., Red Bank, NJ 07701, United ...","[{'label': 'display', 'lat': 40.34828085719662...",40.348281,-74.074357,NaN,07701,NJ,NJT - Red Bank Station (NJCL),v-1549514468,NaN


In [93]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Umberto Restaurant & Pizza,Pizza Place,583 River Rd,US,Fair Haven,United States,NaN,2760,"[583 River Rd, Fair Haven, NJ 07704, United St...","[{'label': 'display', 'lat': 40.3615, 'lng': -...",40.361500,-74.042490,07704,NJ,4cd09ea47b685481f1a3c2f8
1,Urban CoalHouse Pizza and Bar,Pizza Place,2 Bridge Ave,US,Red Bank,United States,NaN,5666,"[2 Bridge Ave, Red Bank, NJ 07701, United States]","[{'label': 'display', 'lat': 40.3503240763069,...",40.350324,-74.075583,07701,NJ,585305d2ac3291412365d68d
2,"MJ's Pizza, Bar & Grill",American Restaurant,1005 State Route 35,US,Middletown,United States,NaN,9171,"[1005 State Route 35, Middletown, NJ 07748, Un...","[{'label': 'display', 'lat': 40.39655171481225...",40.396552,-74.106933,07748,NJ,4d827dc51cfca35dbcbb9781
3,Luigi's Famous Pizza and Ristorante,Pizza Place,86 Oceanport Ave,US,Little Silver,United States,NaN,4640,"[86 Oceanport Ave, Little Silver, NJ 07739, Un...","[{'label': 'display', 'lat': 40.3240136, 'lng'...",40.324014,-74.037609,07739,NJ,4c30b7b716adc928df86bf9c
4,Earth Pizza,Pizza Place,95 Broad St,US,Red Bank,United States,NaN,4831,"[95 Broad St, Red Bank, NJ 07701, United States]","[{'label': 'display', 'lat': 40.34895985603297...",40.348960,-74.065081,07701,NJ,4bb10dcaf964a5203f763ce3
5,MJ's Pizza Bar & Grill,American Restaurant,1213 Sycamore Ave,US,Tinton Falls,United States,at Tinton Ave.,9819,"[1213 Sycamore Ave (at Tinton Ave.), Tinton Fa...","[{'label': 'display', 'lat': 40.30472348521086...",40.304723,-74.100184,07724,NJ,4c7292898efc37046c3b137d
6,Patrizia's Pizza and Pasta,Italian Restaurant,28 Broad St,US,Red Bank,United States,NaN,4824,"[28 Broad St, Red Bank, NJ 07701, United States]","[{'label': 'display', 'lat': 40.35118651772577...",40.351187,-74.065689,07701,NJ,53ebfabd498ee4a59d5b41ad
7,Mangia Brick Oven Pizza - Shrewsbury,Pizza Place,1087 Broad St,US,Shrewsbury,United States,NaN,6606,"[1087 Broad St, Shrewsbury, NJ 07702, United S...","[{'label': 'display', 'lat': 40.3162412, 'lng'...",40.316241,-74.062592,07702,NJ,57ac154c498e96454591240b
8,Mr. Pizza Slice,Pizza Place,10 Monmouth St,US,Red Bank,United States,NaN,4872,"[10 Monmouth St, Red Bank, NJ 07701, United St...","[{'label': 'display', 'lat': 40.34970607300840...",40.349706,-74.065824,07701,NJ,4b50b510f964a520902e27e3
9,Zoni's Coal Fired Pizza,Pizza Place,287 Route 35,US,Middletown,United States,NaN,6280,"[287 Route 35, Middletown, NJ 07701, United St...","[{'label': 'display', 'lat': 40.36682509166925...",40.366825,-74.083499,07701,NJ,5233917211d2c78adfbe0899


As expected, the query surpass the 49 limits of the API, not good. Lets visualize the results, and later decrease some the radio to display more consistent results:

In [11]:
import pandas as pd
import folium
from folium.plugins import FastMarkerCluster, MarkerCluster

df = dataframe_filtered

folium_map = folium.Map(location=[latitude, longitude],
                        zoom_start=13.4,
                        tiles='Stamen Terrain')

mc = MarkerCluster(name="Marker Cluster")

for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=10,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(folium_map)

mc.add_to(folium_map)

folium.LayerControl().add_to(folium_map)
folium_map

Surely its not for sale!
Lets decrease the radio and display the results again...

In [12]:
search_query = 'pizza'
radius = 1000
print('done!')

done!


In [17]:
import pandas as pd
import folium
from folium.plugins import FastMarkerCluster, MarkerCluster

df = dataframe_filtered

folium_map = folium.Map(location=[latitude, longitude],
                        zoom_start=13.4,
                        tiles='Stamen Terrain')

mc = MarkerCluster(name="Marker Cluster")

for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=10,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(folium_map)

mc.add_to(folium_map)

folium.LayerControl().add_to(folium_map)
folium_map

49 results again, showing the overpopulation of places related to pizza, wont even imagine the scenario asking a more generic 'restaurants'

So, lets move around a little. Lets see in another corner of the country, say 7th avenue with 86th street, pretty close with lot of movement also. 

In [52]:
search_query = 'pizza'
radius = 5000
print('done!')

done!


In [23]:
latitude=40.62
longitude=-74.03
print("ready to go!")

ready to go!


In [28]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Espresso Pizza,Pizza Place,9403 5th Ave,US,Brooklyn,United States,NaN,406,"[9403 5th Ave, Brooklyn, NY 11209, United States]","[{'label': 'display', 'lat': 40.61639003656646...",40.616390,-74.030710,11209,NY,4cab2ef644a8224bb2892b40
1,Verrazano Pizza,Pizza Place,9102 4th Ave,US,Brooklyn,United States,NaN,105,"[9102 4th Ave, Brooklyn, NY 11209, United States]","[{'label': 'display', 'lat': 40.61905940061457...",40.619059,-74.029895,11209,NY,4c12dbc4a5eb76b04f02bfb7
2,Pizza Wagon,Pizza Place,8610 5th Ave,US,Brooklyn,United States,86th St,334,"[8610 5th Ave (86th St), Brooklyn, NY 11209, U...","[{'label': 'display', 'lat': 40.62128103447500...",40.621281,-74.026425,11209,NY,4a188c55f964a520c7791fe3
3,Zitelli's Thin Crust Pizza and Bar,Italian Restaurant,8530-3rd Avenue,US,Brooklyn,United States,86st & 3ave Corner,404,"[8530-3rd Avenue (86st & 3ave Corner), Brookly...","[{'label': 'display', 'lat': 40.62355539009036...",40.623555,-74.030969,11209,NY,5071f049e4b05fb6c443ec8e
4,Luigi's Pizza,Pizza Place,8502 3rd Ave,US,Brooklyn,United States,85th St,478,"[8502 3rd Ave (85th St), Brooklyn, NY 11209, U...","[{'label': 'display', 'lat': 40.62425297440284...",40.624253,-74.030784,11209,NY,4b7d7d00f964a520c7c12fe3
5,Artichoke Basille's Pizza,Pizza Place,NaN,US,Brooklyn,United States,NaN,95,"[Brooklyn, NY, United States]","[{'label': 'display', 'lat': 40.61914215222762...",40.619142,-74.029900,NaN,NY,581a921cb293f7196867f9dd
6,Papa's Pizza & Pasta,Pizza Place,8312 5th Ave,US,Brooklyn,United States,NaN,550,"[8312 5th Ave, Brooklyn, NY 11209, United States]","[{'label': 'display', 'lat': 40.6237387, 'lng'...",40.623739,-74.025741,11209,NY,4e4ce15fbd413c4cc66ce99a
7,Krispy Pizza - Brooklyn,Pizza Place,7112 13th Ave,US,Brooklyn,United States,NaN,2092,"[7112 13th Ave, Brooklyn, NY 11228, United Sta...","[{'label': 'display', 'lat': 40.62230976588283...",40.622310,-74.005429,11228,NY,4ba4ff22f964a5200ece38e3
8,Bklyn's Pizza,Pizza Place,1406 86th St,US,Brooklyn,United States,14th ave,1768,"[1406 86th St (14th ave), Brooklyn, NY 11228, ...","[{'label': 'display', 'lat': 40.61248051231578...",40.612481,-74.011565,11228,NY,4be99e48415e20a1020de4bb
9,Nino's Pizza,Pizza Place,9110 3rd Ave,US,Brooklyn,United States,at 91st St,223,"[9110 3rd Ave (at 91st St), Brooklyn, NY 11209...","[{'label': 'display', 'lat': 40.61976265882881...",40.619763,-74.032630,11209,NY,4bb7d799314e9521cacc479d


I wont put all the steps again cause it is a pretty iterative process, and this notebook would get incredibly huge. But above is the result for the new query, 49 results again... lets see them.

In [29]:
import pandas as pd
import folium
from folium.plugins import FastMarkerCluster, MarkerCluster

df = dataframe_filtered

folium_map = folium.Map(location=[latitude, longitude],
                        zoom_start=13.4,
                        tiles='Stamen Terrain')

mc = MarkerCluster(name="Marker Cluster")

for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=10,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(folium_map)

mc.add_to(folium_map)

folium.LayerControl().add_to(folium_map)
folium_map

Ok, totally "overpopulated" area. Lets move around a little. Unfortunately, this process would be much more efficient if the limit of results from Foursquare was not as it is now, but les go ahead. Lets move more to the south, working a little bit with google earth. 

In [30]:
latitude=40.45
longitude=-74.22
print("ready to go!")

ready to go!


In [41]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,V & F Pizza Restaurant,Pizza Place,309A State Route 35,US,Cliffwood,United States,NaN,1354,"[309A State Route 35, Cliffwood, NJ 07721, Uni...","[{'label': 'display', 'lat': 40.43853133038734...",40.438531,-74.225335,07721,NJ,4c083acea1b32d7f6bb895f0
1,Villa Bella Pizza & Restaurant,Pizza Place,NaN,US,Cliffwood,United States,NaN,1549,"[Cliffwood, NJ 07721, United States]","[{'label': 'display', 'lat': 40.43609476011222...",40.436095,-74.220819,07721,NJ,4c51dc097f6e20a1ae5cccea
2,Tony's Pizza & Restaurant,Pizza Place,300 State Route 35,US,Keyport,United States,NaN,1749,"[300 State Route 35, Keyport, NJ 07735, United...","[{'label': 'display', 'lat': 40.43429228004108...",40.434292,-74.219437,07735,NJ,516ed72fe4b0232f1a4b6ab0
3,Keyport Pizza & Restaurant,Pizza Place,42 W Front St,US,Keyport,United States,NaN,2024,"[42 W Front St, Keyport, NJ 07735, United States]","[{'label': 'display', 'lat': 40.436742, 'lng':...",40.436742,-74.203645,07735,NJ,4c2a99842219c928e764a348
4,Ciros Pizza Cafe,Pizza Place,25 Morristown Rd,US,Matawan,United States,Cliffwood Drive,3547,"[25 Morristown Rd (Cliffwood Drive), Matawan, ...","[{'label': 'display', 'lat': 40.427952, 'lng':...",40.427952,-74.250229,07747,NJ,4dc41131d4c0ad9c0f64854d
5,Denino's Pizza Place,Pizza Place,1077 State Route 34,US,Matawan,United States,NaN,5591,"[1077 State Route 34, Matawan, NJ 07747, Unite...","[{'label': 'display', 'lat': 40.4000678, 'lng'...",40.400068,-74.227207,07747,NJ,4b539038f964a5207fa227e3
6,Sal And Ginos Pizza,Pizza Place,216 Broad St,US,Keyport,United States,Maple Ok,2571,"[216 Broad St (Maple Ok), Keyport, NJ 07735, U...","[{'label': 'display', 'lat': 40.43276262226085...",40.432763,-74.199787,07735,NJ,4e59471918388cd5cba17e21
7,Ray's Real Pizza,Italian Restaurant,3429 State Route 35,US,Hazlet,United States,NaN,3547,"[3429 State Route 35, Hazlet, NJ 07730, United...","[{'label': 'display', 'lat': 40.42405197033007...",40.424052,-74.195687,07730,NJ,4d1bbd174ff98cfa1908b53e
8,Giuseppe's Pizza,Pizza Place,620 Beers St,US,Hazlet,United States,NaN,4383,"[620 Beers St, Hazlet, NJ 07730, United States]","[{'label': 'display', 'lat': 40.4145138000593,...",40.414514,-74.197570,07730,NJ,4c7ec394fb74236a5a26f8b9
9,Pizza by kims,None,NaN,US,Keyport,United States,NaN,1210,"[Keyport, NJ 07735, United States]","[{'label': 'display', 'lat': 40.441875, 'lng':...",40.441875,-74.210511,07735,NJ,4cbb412aa33bb1f7983693fd


In [43]:
import pandas as pd
import folium
from folium.plugins import FastMarkerCluster, MarkerCluster

df = dataframe_filtered

folium_map = folium.Map(location=[40.45, -74.22],
                        zoom_start=11.4,
                        tiles='Stamen Terrain')

mc = MarkerCluster(name="Marker Cluster")

for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=10,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(folium_map)

mc.add_to(folium_map)

folium.LayerControl().add_to(folium_map)
folium_map

Ok, so 49 results again. A hot area again. Lets move again, to see if there is a less congested area 

In [58]:
latitude=40.31
longitude=-73.98
print("ready to go!")

ready to go!


In [64]:
import pandas as pd
import folium
from folium.plugins import FastMarkerCluster, MarkerCluster

df = dataframe_filtered

folium_map = folium.Map(location=[40.31, -73.98],
                        zoom_start=11.4,
                        tiles='Stamen Terrain')

mc = MarkerCluster(name="Marker Cluster")

for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=10,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(folium_map)

mc.add_to(folium_map)

folium.LayerControl().add_to(folium_map)
folium_map

Wrong call again. Dont worry, this land somewhere. If one looks closely, the distribution is really dense there. Much less dispersion that in the other cases. So it seems that moving up to say Highlands or similar, can finally give a promising result. 

In [65]:
latitude=40.397
longitude=-73.986
print("ready to go!")

ready to go!


In [71]:
import pandas as pd
import folium
from folium.plugins import FastMarkerCluster, MarkerCluster

df = dataframe_filtered

folium_map = folium.Map(location=[40.397, -73.986],
                        zoom_start=12.4,
                        tiles='Stamen Terrain')

mc = MarkerCluster(name="Marker Cluster")

for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=10,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(folium_map)

mc.add_to(folium_map)

folium.LayerControl().add_to(folium_map)
folium_map

So finally something. For a radio of 5000, just 13 results. We land in a Borough called Highlands, that seems promising. It is near the see, it doesnt seem to be on the desert, and its between really hot spots such as Long Branch as seen before, and Keyport -thee map that is just before Long Branch. It actually seems like Highlands have some movement after all, but Fair Haven, for example, has really not movements. Lets do the final map to see if moving a little bit toward Fair Heaven, reveals more information. Also, it would be nice to get the whole picture of the area. So lets zoom out a little more, and then we can proceed to move toward exploring the place founded.

In [88]:
latitude=40.36
longitude=-74.01
print("ready to go!")

ready to go!


In [96]:
import pandas as pd
import folium
from folium.plugins import FastMarkerCluster, MarkerCluster

df = dataframe_filtered

folium_map = folium.Map(location=[40.36, -74.01],
                        zoom_start=12,
                        tiles='Stamen Terrain')

mc = MarkerCluster(name="Marker Cluster")

for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=10,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(folium_map)

mc.add_to(folium_map)

folium.LayerControl().add_to(folium_map)
folium_map

So, it seems that there are other hot spots, with high density, but Rumson Road remains with no movement at all. Finally, it seems, that this place is totally an option. At least if its not a ghost town or something. It seem that the untoched area is divided between Fair Haven and Rumson. Lets do some research about them.

In [97]:
from lxml import html
import requests

In [98]:
page = requests.get('https://en.wikipedia.org/wiki/Rumson,_New_Jersey')

In [99]:
tree=html.fromstring(page.content)

In [100]:
from bs4 import BeautifulSoup

In [102]:
soup = BeautifulSoup(page.text, 'html.parser')

In [141]:
import requests
from bs4 import BeautifulSoup
 
def count_words(url, the_word):
    r = requests.get(url, allow_redirects=False)
    soup = BeautifulSoup(r.content, 'lxml')
    words = soup.find(text=lambda text: text and the_word in text)
    print(words)
    return len(words)
 
def main():
    url = 'https://en.wikipedia.org/wiki/Rumson,_New_Jersey'
    word = 'income'
    count = count_words(url, word)
    print('\nUrl: {}\ncontains {} occurrences of word: {}'.format(url, count, word))
 
if __name__ == '__main__':
    main()

median household income

Url: https://en.wikipedia.org/wiki/Rumson,_New_Jersey
contains 23 occurrences of word: income


In [ ]:
So, les find manually the relevant words: median household income

In [ ]:
print(soup.prettify())

In [144]:
The median household income was of 134,281 which seems pretty high. Good news. 

In [149]:
import requests
from bs4 import BeautifulSoup
 
def main():
    url = 'https://en.wikipedia.org/wiki/Rumson,_New_Jersey'
    word = 'population'
    count = count_words(url, word)
    
if __name__ == '__main__':
    main()

, the borough's population was 7,122,


So there you go, a population of 7,122 for the community, not much but still close to other locations quite populated. So, lets dig a bit more. But first, we should look a bit more from the county where Rumson belongs, to see, at least, if it is not worst around...  

In [166]:
import requests
from bs4 import BeautifulSoup
 
def main():
    url = 'https://en.wikipedia.org/wiki/Monmouth_County,_New_Jersey'
    word = 'population was'
    count = count_words(url, word)
    
if __name__ == '__main__':
    main()

, the county's population was 626,351, making it the state's fifth-most populous county,


In [170]:
import requests
from bs4 import BeautifulSoup
 
def main():
    url = 'https://en.wikipedia.org/wiki/Monmouth_County,_New_Jersey'
    word = 'income'
    count = count_words(url, word)
    
if __name__ == '__main__':
    main()

per capita personal income


So, one must iterate the process to search for the key word, and see if there is any luck in the process. This systems does not always work, but is worth the try

In [171]:
page = requests.get('https://en.wikipedia.org/wiki/Monmouth_County,_New_Jersey')

In [172]:
soup = BeautifulSoup(page.text, 'html.parser')

In [ ]:
print(soup.prettify())

<a href="/wiki/Per_capita_personal_income_in_the_United_States" title="Per capita personal income in the United States">
        per capita personal income
       </a>
       of $69,410, the fifth-highest in New Jersey and ranked 74th of 3,113 counties in USA
       

So, not bad at all. The county its quite wealthy and Rumson income is even higher ($79,388 per capita). So, it is located in quite a financially healthy place. 

The next step its to work a little on the presentation to be shown to the italian friend. We seem to have a little of luck this time. But lets dig closer with more queries to the relevant spot, to understand more about its infraestructure.

In [177]:
#Ill start the query using a ratio of 2500, expecting the result to surpass the 49 unfortunately limit of the API, using as keyword 'pizza':
search_query = 'hospital'
radius = 5000
print('done!')

done!


In [181]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Fair Haven Animal Hospital,Veterinarian,NaN,US,Fair Haven,United States,2072,"[Fair Haven, NJ, United States]","[{'label': 'display', 'lat': 40.365799, 'lng':...",40.365799,-74.033221,NaN,NJ,514b3927e4b05ed6ecbc417e
1,Hospital For Wayward Children,None,15 Lisa Court,US,NaN,United States,4762,"[15 Lisa Court, New Jersey, United States]","[{'label': 'display', 'lat': 40.337375, 'lng':...",40.337375,-74.057638,NaN,New Jersey,4d143aedbb488cfa0c6695d4
2,Monmouth Medical Hospital,Shop & Service,NaN,US,NaN,United States,3171,"[New Jersey, United States]","[{'label': 'display', 'lat': 40.35266716169504...",40.352667,-73.973869,NaN,New Jersey,4e57ce9da8096558610db2e7
3,Riverview Medical Center,Hospital,1 Riverview Plz,US,Red Bank,United States,4635,"[1 Riverview Plz, Red Bank, NJ 07701, United S...","[{'label': 'display', 'lat': 40.35351484308679...",40.353515,-74.063975,07701,NJ,4b608208f964a5207bea29e3
4,Little Silver Animal Hospital,Veterinarian,NaN,US,Little Silver,United States,4491,"[Little Silver, NJ 07739, United States]","[{'label': 'display', 'lat': 40.32846459215351...",40.328465,-74.043031,07739,NJ,4d0b958b9a443704c09cba67
5,Two Rivers Vet,Veterinarian,457 River Rd,US,Fair Haven,United States,3184,"[457 River Rd, Fair Haven, NJ 07704, United St...","[{'label': 'display', 'lat': 40.35919951666666...",40.359200,-74.047529,07704,NJ,4cbb461ff50e224b81bbf8fb
6,Hospitalty Room #2 - MMC,Hospital,NaN,US,Long Branch,United States,6254,"[Long Branch, NJ 07740, United States]","[{'label': 'display', 'lat': 40.30462914823733...",40.304629,-73.997485,07740,NJ,4f0121b86da1cd035a0e9132
7,Riverview Hospital Dialysis,Medical Center,48 E Front St,US,Red Bank,United States,4596,"[48 E Front St, Red Bank, NJ 07701, United Sta...","[{'label': 'display', 'lat': 40.35266705430916...",40.352667,-74.063325,07701,NJ,4c6bee1b96eaa593007123df


In [182]:
import pandas as pd
import folium
from folium.plugins import FastMarkerCluster, MarkerCluster

df = dataframe_filtered

folium_map = folium.Map(location=[40.36, -74.01],
                        zoom_start=12,
                        tiles='Stamen Terrain')

mc = MarkerCluster(name="Marker Cluster")

for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=10,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(folium_map)

mc.add_to(folium_map)

folium.LayerControl().add_to(folium_map)
folium_map

In [183]:
#Ill start the query using a ratio of 2500, expecting the result to surpass the 49 unfortunately limit of the API, using as keyword 'pizza':
search_query = 'school'
radius = 5000
print('done!')

done!


In [187]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,The Rumson Country Day School,Private School,NaN,US,Rumson,United States,NaN,398,"[Rumson, NJ, United States]","[{'label': 'display', 'lat': 40.36319949841999...",40.363199,-74.012107,NaN,NJ,552c2bb3498ea8f3dc9052fd
1,Rumson Fair Haven High School,High School,NaN,US,Rumson,United States,NaN,1213,"[Rumson, NJ 07760, United States]","[{'label': 'display', 'lat': 40.36968056679983...",40.369681,-74.003425,07760,NJ,4b6a0358f964a520c2c12be3
2,Rumson High School,None,NaN,US,Rumson,United States,NaN,1181,"[Rumson, NJ 07760, United States]","[{'label': 'display', 'lat': 40.368867, 'lng':...",40.368867,-74.002341,07760,NJ,4c1bfa2f8b3aa593e720985f
3,School of Rock - Red Bank,Music School,52 Monmouth St,US,Red Bank,United States,NaN,5035,"[52 Monmouth St, Red Bank, NJ 07701, United St...","[{'label': 'display', 'lat': 40.349648, 'lng':...",40.349648,-74.067788,07701,NJ,4f160b11e4b00420590258ea
4,Forrestdale School,School,Forrest Ave,US,Rumson,United States,NaN,1269,"[Forrest Ave, Rumson, NJ 07760, United States]","[{'label': 'display', 'lat': 40.37033655294107...",40.370337,-74.003668,07760,NJ,4bbf9b21abf495215cefbd93
5,Rumson Fair haven high school auditorium,Auditorium,NaN,US,Rumson,United States,NaN,1305,"[Rumson, NJ 07760, United States]","[{'label': 'display', 'lat': 40.3714702, 'lng'...",40.371470,-74.006766,07760,NJ,4f876dc6e4b09582a083675b
6,Red Bank Regional High School,High School,101 Ridge Rd,US,Little Silver,United States,NaN,2749,"[101 Ridge Rd, Little Silver, NJ 07739, United...","[{'label': 'display', 'lat': 40.35016004059454...",40.350160,-74.039729,07739,NJ,4be16a84d816c928d5f7efd9
7,Knollwood School,College Academic Building,Hance Street,US,Fair Haven,United States,NaN,2803,"[Hance Street, Fair Haven, NJ 07704-3523, Unit...","[{'label': 'display', 'lat': 40.35620925602536...",40.356209,-74.042678,07704-3523,NJ,4c8fccb4352da1cde5540cb2
8,Sickles Elementary School,Elementary School,NaN,US,Fair Haven,United States,NaN,2015,"[Fair Haven, NJ, United States]","[{'label': 'display', 'lat': 40.36374963449813...",40.363750,-74.033251,NaN,NJ,4d8cc258cb9b224b134f5c41
9,Point Road Elementary School,Elementary School,357 Little Silver Point Rd,US,Little Silver,United States,at Seven Bridges Rd,3433,[357 Little Silver Point Rd (at Seven Bridges ...,"[{'label': 'display', 'lat': 40.33403692724303...",40.334037,-74.031851,07739,NJ,4d9dc8d9b0fccbfff44f3ccf


In [188]:
import pandas as pd
import folium
from folium.plugins import FastMarkerCluster, MarkerCluster

df = dataframe_filtered

folium_map = folium.Map(location=[40.36, -74.01],
                        zoom_start=12,
                        tiles='Stamen Terrain')

mc = MarkerCluster(name="Marker Cluster")

for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=10,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(folium_map)

mc.add_to(folium_map)

folium.LayerControl().add_to(folium_map)
folium_map

In [189]:
#Ill start the query using a ratio of 2500, expecting the result to surpass the 49 unfortunately limit of the API, using as keyword 'pizza':
search_query = 'bank'
radius = 5000
print('done!')

done!


So, lots of schools and hospital arounds. As seen before, sourrunded by very hot spots, and at the same time, with lots of potencial consumers with quite a wealthy situation. Seems like a safe bet. 